<a href="https://colab.research.google.com/github/nityanandkore/6156-ML-Final-Project/blob/main/Final_Project_04_NK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style='color:green'>Modeling Airline Flight Delays</span>

In [26]:
# import libraries
import os
from IPython import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import Ridge, LinearRegression, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, roc_curve
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [27]:
#This will allow me to avoid typing out the full file path each time I use read_csv
import os

a_path = "C:\\<PATH>"
a_folder = "Data"
a_file = "IMG_DataCollecting.jpg" # https://drive.google.com/file/d/1PHSosWV4_coJ7uac7HBe6nVWY1IvzQD4/view?usp=sharing
joined_path = os.path.join(a_path, a_folder, a_file)

print(joined_path)

C:\<PATH>/Data/IMG_DataCollecting.jpg


# I. Data Importing

In [28]:
#from IPython import display
#display.Image(joined_path)


![picture](https://drive.google.com/uc?id=1PHSosWV4_coJ7uac7HBe6nVWY1IvzQD4)

In [29]:
# import FAA and summary datasets
#joined_path = os.path.join(a_path, a_folder, "FAA_aircraft_details.csv")  # https://drive.google.com/file/d/1LBKAU9JwmEc5UmDLWEOq-2Ntm9TXEOsZ/view?usp=sharing
#aircraft = pd.read_csv(joined_path)
url = 'https://drive.google.com/uc?id=1LBKAU9JwmEc5UmDLWEOq-2Ntm9TXEOsZ'
aircraft = pd.read_csv(url)

#joined_path = os.path.join(a_path, a_folder, "airline_carrier_volume.csv") # https://drive.google.com/file/d/1MUWu0da02vQCWt7P0D21JAvA5a280XBL/view?usp=sharing
#airline_vol = pd.read_csv(joined_path)
url= 'https://drive.google.com/uc?id=1MUWu0da02vQCWt7P0D21JAvA5a280XBL'
airline_vol = pd.read_csv(url)

#joined_path = os.path.join(a_path, a_folder, "airport_nodes_2018.csv") # https://drive.google.com/file/d/1xNqu30SZjrcV_J8eWpZa20_m-ViESdXd/view?usp=sharing
#airports_2018 = pd.read_csv(joined_path)
url= 'https://drive.google.com/uc?id=1xNqu30SZjrcV_J8eWpZa20_m-ViESdXd'
airports_2018 = pd.read_csv(url)

#joined_path = os.path.join(a_path, a_folder, "airport_nodes_2019.csv") # https://drive.google.com/file/d/1gioGAw-7WieA4bQ3-el8PPD85KcF83T3/view?usp=sharing
#airports_2019 = pd.read_csv(joined_path)
url='https://drive.google.com/uc?id=1gioGAw-7WieA4bQ3-el8PPD85KcF83T3'
airports_2019 = pd.read_csv(url)

#joined_path = os.path.join(a_path, a_folder, "edges_2018.csv")  # https://drive.google.com/file/d/1TRsaIC04goKkCnWaDkuIo25etTt4QEQC/view?usp=sharing
#airport_connections_2018 = pd.read_csv(joined_path)
url= 'https://drive.google.com/uc?id=1TRsaIC04goKkCnWaDkuIo25etTt4QEQC'
airport_connections_2018 = pd.read_csv(url)

#joined_path = os.path.join(a_path, a_folder, "edges_2019.csv") # https://drive.google.com/file/d/1XXQEAHLaZ0ERkHIIAzV3wrmlQ4a8Fhug/view?usp=sharing
#airport_connections_2019 = pd.read_csv(joined_path)
url='https://drive.google.com/uc?id=1XXQEAHLaZ0ERkHIIAzV3wrmlQ4a8Fhug'
airport_connections_2019 = pd.read_csv(url)

#joined_path = os.path.join(a_path, a_folder, "Airline_Codes.csv") # https://drive.google.com/file/d/1esJ93wv0vfIdXf3UsExXDvv90Zl4s-Bp/view?usp=sharing
#airline_codes = pd.read_csv(joined_path,encoding='latin1')
url= 'https://drive.google.com/uc?id=1esJ93wv0vfIdXf3UsExXDvv90Zl4s-Bp'
airline_codes = pd.read_csv(url,encoding='latin1')


print('TABLE                    (ROWS, COLUMNS)')
print('')
print('aircraft:               ', aircraft.shape)
print('airline volume:         ', airline_vol.shape)
print('airports 2018:          ', airports_2018.shape)
print('airports 2019:          ', airports_2019.shape)
print('airport connects 2018:  ', airport_connections_2018.shape)
print('airport connects 2019:  ', airport_connections_2019.shape)
print('airline codes:          ', airline_codes.shape)

TABLE                    (ROWS, COLUMNS)

aircraft:                (6603, 55)
airline volume:          (28, 14)
airports 2018:           (371, 12)
airports 2019:           (373, 12)
airport connects 2018:   (6454, 4)
airport connects 2019:   (6658, 4)
airline codes:           (1652, 2)


### Build Dataset (based on 2019 data)

In [35]:
# import 2019 flights (very large dataset)
#joined_path = os.path.join(a_path, a_folder, "flights_all_2019.csv") # https://drive.google.com/file/d/1pPqmFyvOLPbBEr1zwjraxJwJfD1krGl3/view?usp=sharing
#flights2019 = pd.read_csv(joined_path)
url= 'https://drive.google.com/uc?id=1pPqmFyvOLPbBEr1zwjraxJwJfD1krGl3'  
flights2019 = pd.read_csv(url)

print('flights 2019 table dimensions: ', flights2019.shape)

flights 2019 table dimensions:  (1, 5)


Above Option didnt work so followed steps mentioned at: https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/

In [36]:
!pip install PyDrive

In [37]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [39]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Authorize with your Google ID, and paste in the link that comes up and press Enter!

In [40]:
downloaded = drive.CreateFile({'id':"1pPqmFyvOLPbBEr1zwjraxJwJfD1krGl3"})   # replace the id with id of file you want to access
downloaded.GetContentFile('flights_all_2019.csv')        # replace the file name with your file

In [41]:
flights2019 = pd.read_csv('flights_all_2019.csv')

print('flights 2019 table dimensions: ', flights2019.shape)

flights 2019 table dimensions:  (8091684, 44)


In [42]:
flights2019.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,DUP,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DISTANCE_GROUP,FL_NUM_COMBO,FL_NUM_COMBO_FULL,ELAPSED_FROM_SCHED_DEP,ELAPSE_RATIO
0,2019,1,1,11,5,2020-01-11,WN,19393,WN,N220WN,1480,13204,1320402,MCO,"Orlando, FL",FL,14122,1412202,PIT,"Pittsburgh, PA",PA,920,918.0,-2.0,9.0,927.0,1125.0,6.0,1135,1131.0,-4.0,0,NaN,0,N,135.0,133.0,118.0,834,4,WN 1480,1/11/2019 WN 1480 MCO to PIT,131.0,0.985185
1,2019,1,1,11,5,2020-01-11,WN,19393,WN,N7889A,2199,13204,1320402,MCO,"Orlando, FL",FL,14122,1412202,PIT,"Pittsburgh, PA",PA,1420,1422.0,2.0,14.0,1436.0,1635.0,5.0,1645,1640.0,-5.0,0,NaN,0,N,145.0,138.0,119.0,834,4,WN 2199,1/11/2019 WN 2199 MCO to PIT,140.0,0.951724
2,2019,1,1,11,5,2020-01-11,WN,19393,WN,N764SW,402,13204,1320402,MCO,"Orlando, FL",FL,14307,1430705,PVD,"Providence, RI",RI,1300,1259.0,-1.0,9.0,1308.0,1529.0,3.0,1545,1532.0,-13.0,0,NaN,0,N,165.0,153.0,141.0,1072,5,WN 402,1/11/2019 WN 402 MCO to PVD,152.0,0.927273
3,2019,1,1,11,5,2020-01-11,WN,19393,WN,N8309C,458,13204,1320402,MCO,"Orlando, FL",FL,14307,1430705,PVD,"Providence, RI",RI,2015,2010.0,-5.0,15.0,2025.0,2249.0,2.0,2255,2251.0,-4.0,0,NaN,0,N,160.0,161.0,144.0,1072,5,WN 458,1/11/2019 WN 458 MCO to PVD,156.0,1.006250
4,2019,1,1,11,5,2020-01-11,WN,19393,WN,N8686A,1966,13204,1320402,MCO,"Orlando, FL",FL,14307,1430705,PVD,"Providence, RI",RI,755,753.0,-2.0,10.0,803.0,1025.0,3.0,1035,1028.0,-7.0,0,NaN,0,N,160.0,155.0,142.0,1072,5,WN 1966,1/11/2019 WN 1966 MCO to PVD,153.0,0.968750


In [43]:
# Create a sample to work with until you're ready to use the entire dataset
sample2019 = flights2019.sample(n=750000, random_state=123)

print(sample2019.shape)

(750000, 44)


In [44]:
# TO DO: Need to switch 'df' from the sample2019 table to the full table if you want to use the enture dataset


# df will reference either the full data set or the sample of that dataset. 
# This way we can simply change this cell when we are ready to switch from the sample to the full dataset.
df = sample2019
# df = flights2019

In [45]:
# combine datasets into a single base table
# Note: airports_2018 & airport_connections_2019 were used to get airport attributes from the prior year
combo_2019 = df.merge(aircraft, 
                      on='TAIL_NUM',
                      how='left').merge(airports_2018,
                                        on=['ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR'],
                                        how='left').merge(airport_connections_2018,
                                                          on=['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID'],
                                                          how='left').merge(airline_vol, 
                                                                            on="OP_UNIQUE_CARRIER",
                                                                            how='left')

print(combo_2019.shape)

(750000, 121)


In [46]:
# Make a new dataframe object that is a copy of our dataset.
# This will allow us to experiment with making changes without changing the original dataset.
BASE_TABLE = combo_2019.copy()

# Create a variable to hold the number of rows in the BASE_TABLE for later reference
n = BASE_TABLE.shape[0]
print("n = ", n)

n =  750000


In [47]:
# Some Aircraft Manufactures have more than one name in the dataset. Create consistent naming.

# print(combo_2019.groupby('MFR_Aircraft').size())
# print("")

# COMMENT OUT THIS CODE. There is already a column in the imported table called ACFT_MFR_DeDup that accomplishes this

# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('ALENIA', case=False), "MFR_Aircraft"] = "ALENIA"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('AIRBUS', case=False), "MFR_Aircraft"] = "AIRBUS"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('BOEING', case=False), "MFR_Aircraft"] = "BOEING"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('DIAMOND AIRCRAFT IND', case=False), "MFR_Aircraft"] = "DIAMOND AIRCRAFT IND"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('EMBRAER', case=False), "MFR_Aircraft"] = "EMBRAER"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('MCDONNELL DOUGLAS', case=False), "MFR_Aircraft"] = "MCDONNELL DOUGLAS"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('MOONEY', case=False), "MFR_Aircraft"] = "MOONEY"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft.astype(str).str.contains('ROBINSON HELICOPTER CO', case=False), "MFR_Aircraft"] = "ROBINSON HELICOPTER CO"
# BASE_TABLE.loc[BASE_TABLE.MFR_Aircraft == 'DOUGLAS', "MFR_Aircraft"] = "MCDONNELL DOUGLAS"

# print(BASE_TABLE.groupby('ACFT_MFR_DeDup').size())

In [48]:
[ i for i in BASE_TABLE.columns]

['YEAR',
 'QUARTER',
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'FL_DATE',
 'OP_UNIQUE_CARRIER',
 'OP_CARRIER_AIRLINE_ID',
 'OP_CARRIER',
 'TAIL_NUM',
 'OP_CARRIER_FL_NUM',
 'ORIGIN_AIRPORT_ID',
 'ORIGIN_AIRPORT_SEQ_ID',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'ORIGIN_STATE_ABR',
 'DEST_AIRPORT_ID',
 'DEST_AIRPORT_SEQ_ID',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_ABR',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY',
 'CANCELLED',
 'CANCELLATION_CODE',
 'DIVERTED',
 'DUP',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'DISTANCE_GROUP',
 'FL_NUM_COMBO',
 'FL_NUM_COMBO_FULL',
 'ELAPSED_FROM_SCHED_DEP',
 'ELAPSE_RATIO',
 'N_Number',
 'SOURCE2',
 'SERIAL_NUMBER',
 'MFR_MDL_CODE',
 'ENG_MFR_MDL',
 'YEAR_MFR',
 'TYPE_REGISTRANT',
 'NAME',
 'STREET',
 'STREET2',
 'CITY',
 'STATE',
 'ZIP_CODE',
 'REGION',
 'COUNTY',
 'COUNTRY',
 'LAST_ACTION_DATE',
 'CERT_ISSUE_DATE',
 'CERTIFICATIO

In [49]:
# Some Aircraft Models also need consistent naming


# COMMENT OUT THIS CODE. This was already accomplished in the base table that was imported

# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='ATR 42-500', "MODEL_Aircraft"] = "ATR42-500"
# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='DC-9-83 (MD-83)', "MODEL_Aircraft"] = "DC-9-83(MD-83)"
# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='DC-9-83', "MODEL_Aircraft"] = "DC-9-83(MD-83)"
# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='MD-83', "MODEL_Aircraft"] = "DC-9-83(MD-83)"
# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='MD 83', "MODEL_Aircraft"] = "DC-9-83(MD-83)"
# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='SAAB 2000', "MODEL_Aircraft"] = "2000"
# BASE_TABLE.loc[BASE_TABLE.MODEL_Aircraft=='SAAB 340B', "MODEL_Aircraft"] = "340B"

In [ ]:
# Create a column to hold the aircraft model group

BASE_TABLE['MODEL_GROUP_Aircraft'] = BASE_TABLE['MODEL_Aircraft'].astype(str)

BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('737', case=False), "MODEL_GROUP_Aircraft"] = "737"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('757', case=False), "MODEL_GROUP_Aircraft"] = "757"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('767', case=False), "MODEL_GROUP_Aircraft"] = "767"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('777', case=False), "MODEL_GROUP_Aircraft"] = "777"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('787', case=False), "MODEL_GROUP_Aircraft"] = "787"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('A319', case=False), "MODEL_GROUP_Aircraft"] = "A319"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('A320', case=False), "MODEL_GROUP_Aircraft"] = "A320"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('A321', case=False), "MODEL_GROUP_Aircraft"] = "A321"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('CL-600', case=False), "MODEL_GROUP_Aircraft"] = "CL-600"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('DC-9', case=False), "MODEL_GROUP_Aircraft"] = "DC-9"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('DHC-8', case=False), "MODEL_GROUP_Aircraft"] = "DHC-8"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('EMB-145', case=False), "MODEL_GROUP_Aircraft"] = "EMB-145"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('ERJ 170', case=False), "MODEL_GROUP_Aircraft"] = "ERJ 170"
BASE_TABLE.loc[BASE_TABLE.MODEL_GROUP_Aircraft.astype(str).str.contains('ERJ 190', case=False), "MODEL_GROUP_Aircraft"] = "ERJ 190"

print("10 most common aircraft model groups:")
print(BASE_TABLE.groupby('MODEL_GROUP_Aircraft').size().sort_values(ascending=False)[0:10])

In [ ]:
# Some engine manufacturers need consistent naming

# print(BASE_TABLE.MFR_Engine.value_counts())
# print("")

BASE_TABLE.loc[BASE_TABLE.MFR_Engine.astype(str).str.contains('CFM INTL', case=False), "MFR_Engine"] = "CFM INTL"
BASE_TABLE.loc[BASE_TABLE.MFR_Engine.astype(str).str.contains('P & W', case=False), "MFR_Engine"] = "P&W"
BASE_TABLE.loc[BASE_TABLE.MFR_Engine.astype(str).str.contains('P&W CANADA', case=False), "MFR_Engine"] = "P&W"
BASE_TABLE.loc[BASE_TABLE.MFR_Engine.astype(str).str.contains('ROLLS', case=False), "MFR_Engine"] = "ROLLS-ROYC"

print(BASE_TABLE.MFR_Engine.value_counts())

In [ ]:
# Export the combined base table to csv

# This has been done. Commenting out below code to avoid accidental overwrite. 

# joined_path = os.path.join(a_path, a_folder, "COMBINED_TABLE_2019.csv")
# BASE_TABLE.to_csv(joined_path, index=False)

________________________________________________________________________________

# II. Data Exploration

In [ ]:
#display.Image(os.path.join(a_path, a_folder, "IMG_Explorer.jpg")) # https://drive.google.com/file/d/1rE1-JbMCeoR6hEXeZV36KW1jnCTrDFb_/view?usp=sharing

![picture](https://drive.google.com/uc?id=1rE1-JbMCeoR6hEXeZV36KW1jnCTrDFb_)

In [ ]:
# If opening a new session with this notebook, you can import the combo dataset instead of running the above code.
# Note: Importing the full dataset takes a very long time. It's better to just run the code above.
# To import, uncomment the lines of code below:

# joined_path = os.path.join(a_path, a_folder, "COMBINED_TABLE_2018.csv")
# BASE_TABLE = pd.read_csv(joined_path, low_memory=False)

In [ ]:
# Create a working copy of the BASE_TABLE to make further transformations to
basecopy = BASE_TABLE.copy()

In [ ]:
# Target Variable Source Column: DEP_DELAY

# Potention Predictor Variable Source Columns:

# QUARTER     
# MONTH     
# DAY_OF_MONTH     
# DAY_OF_WEEK     
# OP_CARRIER     
# ORIGIN     
# ORIGIN_CITY_NAME (ignore)   
# ORIGIN_STATE_ABR (ignore)
# DEST             (ignore)
# DEST_CITY_NAME   (ignore)
# DEST_STATE_ABR   (ignore)
# CRS_DEP_TIME                    (numeric)
# DISTANCE                        (numeric)
# DISTANCE_GROUP
# YEAR_MFR                        (numeric)
# TYPE_AIRCRAFT
# TYPE_ENGINE
# NO_ENG                          (numeric)
# NO_SEATS                        (numeric)
# AC_WEIGHT
# MFR_Engine
# THRUST
# ACFT_MFR_DeDup
# ENGREF_MATCHES                  (as a filtering column if engine columns are significant)
# annual_departure_count          (numeric)
# median_monthly_dep_cnt          (numeric)
# mean_monthly_dep_cnt            (numeric)
# airline_count                   (numeric)
# annual_number_of_flights_btwn   (numeric)
# distance_btwn                   (numeric)
# ACTIVE_AIRCRAFT_CNT_2018        (numeric)   
# ANNUAL_FLT_CNT_2018             (numeric)
# ANNUAL_AIR_MILES_2018           (numeric)
# MODEL_GROUP_Aircraft

In [ ]:
# *************** MAKE BINARY TARGET COLUMNS ***************

# Create a binary versions of the target variable
basecopy["DEP_DELAY_GE15"] = (basecopy["DEP_DELAY"]>=15).astype(int)
basecopy["DEP_DELAY_GE30"] = (basecopy["DEP_DELAY"]>=30).astype(int)
basecopy["DEP_DELAY_GE45"] = (basecopy["DEP_DELAY"]>=45).astype(int)
basecopy["DEP_DELAY_GE60"] = (basecopy["DEP_DELAY"]>=60).astype(int)

# Also create a version of the target column that is in hours instead of minutes
basecopy["DEP_DELAY_HR"] = round(basecopy["DEP_DELAY"]/60, 2)


# Confirm that the new columns look correct:
# print(basecopy.loc[basecopy.DEP_DELAY>-10,
#                    ["DEP_DELAY_HR", "DEP_DELAY", "DEP_DELAY_GE30", "DEP_DELAY_GE45", "DEP_DELAY_GE60"]].head(10))

In [ ]:
#basecopy.to_csv(joined_path, index=False)

In [ ]:
np.random.uniform(size=basecopy.shape[0])

In [ ]:
# *************** TRAIN/TEST SPLIT ***************

# Keep 60% for training, 20% for validation, and 20% for backtesting after model is completed.
# We are doing this at the beginning of the analysis phase so that any insights we use are for the training set only.
np.random.seed(123)
basecopy["randuni"]=np.random.uniform(size=basecopy.shape[0])

training = basecopy[basecopy.randuni<=0.60].copy()
validation = basecopy[(basecopy.randuni>0.60) & (basecopy.randuni<=0.80)].copy()
backtest = basecopy[basecopy.randuni>0.80].copy()

print("training pct:   ", 100*training.shape[0]/n , "%")
print("validation pct: ", 100*validation.shape[0]/n , "%")
print("backtest pct:   ", 100*backtest.shape[0]/n , "%")
print("")
print(training.shape[0], " + ", validation.shape[0], " + ", backtest.shape[0], " = ",
      training.shape[0] + validation.shape[0] + backtest.shape[0])
print("")
print("Confirm that the percent delayed is similar for all three datasets:")
print("training delay rate:   ", round(100*training[training.DEP_DELAY>=30].shape[0]/training.shape[0], 1), "%")
print("validation delay rate: ", round(100*validation[validation.DEP_DELAY>=30].shape[0]/validation.shape[0], 1), "%")
print("backtest delay rate:   ", round(100*backtest[backtest.DEP_DELAY>=30].shape[0]/backtest.shape[0], 1), "%")

#### Target Variable Exploration

In [ ]:
# Dependent Variable

# Distribution
print("DISTRIBUTION OF DEP_DELAY")
print(training["DEP_DELAY"].describe())
print("")
print("")

print("At least 15min delay row count: ", training[training.DEP_DELAY>=15].shape[0])
print("At least 15min delay row count: ", round(100*training[training.DEP_DELAY>10].shape[0]/training.shape[0], 1), "%")
print("")
print("At least 30min delay row count: ", training[training.DEP_DELAY>=30].shape[0])
print("At least 30min delay frequency: ", round(100*training[training.DEP_DELAY>=30].shape[0]/training.shape[0], 1), "%")
print("")
print("At least 45min delay row count: ", training[training.DEP_DELAY>=45].shape[0])
print("At least 45min delay frequency: ", round(100*training[training.DEP_DELAY>=45].shape[0]/training.shape[0], 1), "%")
print("")
print("At least 60min delay row count: ", training[training.DEP_DELAY>=60].shape[0])
print("At least 60min delay frequency: ", round(100*training[training.DEP_DELAY>=60].shape[0]/training.shape[0], 1), "%")

In [ ]:
# Plot a histogram of target distribution
training["DEP_DELAY"].hist(bins='auto')
plt.title("Distribution of Target Variable")
plt.xlabel("target value")
plt.ylabel("count")
print("")

In [ ]:
np.log(training[training.DEP_DELAY>0]["DEP_DELAY"]).hist(bins='auto')
plt.title("Distribution of Target Variable")
plt.xlabel("natural log of target value")
plt.ylabel("count")
print("")

#### Numeric Predictor Exploration

In [ ]:
# CRS_DEP_TIME
# The last two digits of the departure time are the minutes. 

print("Min Dep Time: ", np.min(training.CRS_DEP_TIME))
print("Max Dep Time: ", np.max(training.CRS_DEP_TIME))

In [ ]:
training[['DISTANCE', 'NO_SEATS', 'annual_departure_count', 'airline_count', 'annual_number_of_flights_btwn', 
          'ACTIVE_AIRCRAFT_CNT_2018', 'ANNUAL_FLT_CNT_2018', 'ANNUAL_AIR_MILES_2018']].describe()


# NOTE: Half the airports have more than 136K flights a year according to annual_departure_count 

In [ ]:
# Take a small random sample for faster graphing
trainingSample = training.sample(n=10000, random_state=123)

In [ ]:
# plot scatterplots of numeric variables against the target variable

numerics_X = trainingSample[['DISTANCE', 'NO_SEATS', 'annual_departure_count', 'airline_count', 
                     'annual_number_of_flights_btwn', 'ACTIVE_AIRCRAFT_CNT_2018', 'ANNUAL_FLT_CNT_2018', 
                     'ANNUAL_AIR_MILES_2018']]

numerics_y = trainingSample["DEP_DELAY"]

fig, axes = plt.subplots(4,2, figsize=(20,10))

for i, ax in enumerate(axes.ravel()):
    ax.plot(numerics_X.iloc[:,i], numerics_y, 'o', alpha=.5)
    ax.set_title("{}: {}".format(i, numerics_X.columns[i]), weight='bold', size=20)
    ax.set_ylabel("Target", weight='bold')
    
fig.tight_layout(pad=3.0)

plt.show()

In [ ]:
print("RIGHT-SKEWED")

# Plot a histogram of different variable distributions
trainingSample["DISTANCE"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
np.log(trainingSample["DISTANCE"]).hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
trainingSample["NO_SEATS"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
np.log(trainingSample["NO_SEATS"]+1).hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
trainingSample["annual_departure_count"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
trainingSample["airline_count"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
print("RIGHT-SKEWED")

trainingSample["annual_number_of_flights_btwn"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
np.log(trainingSample["annual_number_of_flights_btwn"]).hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
trainingSample["ACTIVE_AIRCRAFT_CNT_2018"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
trainingSample["ANNUAL_FLT_CNT_2018"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
np.log(trainingSample["ANNUAL_FLT_CNT_2018"]).hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
trainingSample["ANNUAL_AIR_MILES_2018"].hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

In [ ]:
np.log(trainingSample["ANNUAL_AIR_MILES_2018"]).hist(bins='auto')
plt.title("Distribution of Variable")
plt.xlabel("variable value")
plt.ylabel("count")
print("")

#### Categorical Predictor Exploration

In [ ]:
# How many rows remain if we only keep rows where ENGREF_MATCHES==True? 
print(basecopy[basecopy.ENGREF_MATCHES==True].shape)
print(basecopy[basecopy.ENGREF_MATCHES==False].shape)
print(basecopy.shape[0])
print(basecopy[basecopy.ENGREF_MATCHES==True].shape[0] + basecopy[basecopy.ENGREF_MATCHES==False].shape[0])

#### Use crosstab frequency tables to see if categories have higher than avg delay percentage

In [ ]:
print("VARIABLE: NO_ENG")
print("________________")
print("")
print("Value Counts:")
print(training.NO_ENG.value_counts())
print("")

cxtab = pd.crosstab(training.NO_ENG, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print("Crosstab Freqs:")
print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: This variable probably not valuable.

In [ ]:
print("VARIABLE: QUARTER")
print("________________")
print("")
print("Value Counts:")
print(training.QUARTER.value_counts())
print("")

cxtab = pd.crosstab(training.QUARTER, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print("Crosstab Freqs:")
print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: Variable not very informative

###  <span style='color:blue'> * Potentially Useful Column Found: MONTH * </span>

In [ ]:
print("VARIABLE: MONTH")
print("________________")
print("")
print("Value Counts:")
print(training.MONTH.value_counts())
print("")

cxtab = pd.crosstab(training.MONTH, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print("Crosstab Freqs:")
print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: Use this variable. Months 2,6,7 are higher than avg and 9,10,11 are lower than avg

In [ ]:
print("VARIABLE: DAY_OF_MONTH")
print("________________")
# print("")
# print("Value Counts:")
# print(training.DAY_OF_MONTH.value_counts())
print("")

cxtab = pd.crosstab(training.DAY_OF_MONTH, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print("Crosstab Freqs:")
print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: I was hoping to see some pattern like the end of the month is higher delays or something, but none exits.

###  <span style='color:blue'> * Potentially Useful Column Found: DAY_OF_WEEK * </span>

In [ ]:
# NOTE: 1 is Monday, 7 is Sunday
print("VARIABLE: DAY_OF_WEEK")
print("________________")
print("")
print("Value Counts:")
print(training.DAY_OF_WEEK.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.DAY_OF_WEEK, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: Mondays, Thursdays, and Fridays have the most airtraffic by counts, and have the highest delay percents
#             This can be used as a proxy for how busy an airport might be on a given day

###  <span style='color:blue'> * Potentially Useful Column Found: OP_CARRIER * </span>

In [ ]:
print("VARIABLE: OP_CARRIER")
print("________________")
print("")
print("Value Counts:")
print(training.OP_CARRIER.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.OP_CARRIER, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: There are some carriers with much higher and much lower than avg delay percentages. Likely useful column.
#             Will likely need to combine some categories to reduce dimensionality.

In [ ]:
# Get the avg delay for each airline
carrier_avg_delays = basecopy.loc[basecopy.DEP_DELAY>=15, ['OP_CARRIER',
                                                          'DEP_DELAY']].groupby(by='OP_CARRIER', 
                                                                                as_index=False).mean().rename(columns={'DEP_DELAY':'AVG_DELAY'})


carrier_pct_delayed = basecopy[['OP_CARRIER', 
                                'DEP_DELAY_GE15', 
                                'DEP_DELAY_GE60', 
                                'MONTH']].groupby(by='OP_CARRIER', 
                                                  as_index=False).agg({'DEP_DELAY_GE15':'sum', 
                                                                       'DEP_DELAY_GE60':'sum',
                                                                       'MONTH':'count'}).rename(columns={'MONTH':'TOTAL_FLIGHTS'})


# Get the % of flights delays for each airline (as well as number of flights and % of flights delayed >=60)
carrier_pct_delayed['PCT_DELAYED']=100*carrier_pct_delayed.DEP_DELAY_GE15/carrier_pct_delayed.TOTAL_FLIGHTS
carrier_pct_delayed['PCT_GE60']=100*carrier_pct_delayed.DEP_DELAY_GE60/carrier_pct_delayed.TOTAL_FLIGHTS
carrier_pct_delayed = carrier_pct_delayed[['OP_CARRIER', 'PCT_DELAYED', 'PCT_GE60', 'TOTAL_FLIGHTS']]


# Merge the datasets (and bring in the airline name to replace the code)
carrier_stats = carrier_avg_delays.merge(carrier_pct_delayed).merge(airline_codes).sort_values(by=['TOTAL_FLIGHTS'])
carrier_stats = carrier_avg_delays.merge(carrier_pct_delayed).merge(airline_codes)
carrier_stats = carrier_stats.loc[carrier_stats.TOTAL_FLIGHTS>7000].reset_index().drop('index', axis=1)

carrier_stats['AIRLINE'] = carrier_stats.AIRLINE.str.split().str.get(0)
carrier_stats.loc[carrier_stats.AIRLINE=='Trans', 'AIRLINE'] = 'Trans States'
carrier_stats.loc[carrier_stats.AIRLINE=='Air', 'AIRLINE'] = 'Air WI'
carrier_stats.loc[carrier_stats.AIRLINE=='Allegiant', 'AIRLINE'] = 'Allgnt'
print(carrier_stats)

In [ ]:

# Plot the airlines to see which ones had the worst/best delay record
fig, ax = plt.subplots(figsize=(20,12))
ax.scatter(carrier_stats['PCT_DELAYED'], carrier_stats['AVG_DELAY'], 
           alpha=0.7, marker=",", s=carrier_stats['TOTAL_FLIGHTS']/200)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('2019 Delays by Airline', weight='semibold', size=25)
plt.xlabel('% of Flights Delayed', weight='medium', size=20)
plt.ylabel('Avg Delay (mins)', weight='medium', size=20)
for i, label in enumerate(carrier_stats['AIRLINE']):
    ax.text(carrier_stats['PCT_DELAYED'][i]-0.3, carrier_stats['AVG_DELAY'][i]+.8, label,
           weight='semibold', size=15)
ax.text(8.7, 96, 'Square size denotes ', size=15, weight='bold', color='#1f77b4')
ax.text(8.9, 94, 'number of flights ', size=15, weight='bold', color='#1f77b4')
plt.show()


# plt.figure(figsize=(20,10))
# plt.scatter(carrier_stats['PCT_DELAYED'],carrier_stats['AVG_DELAY'],
#             marker=",", s=carrier_stats['TOTAL_FLIGHTS']/200)
# plt.xlabel("X")
# plt.ylabel("Y")
# plt.title("Scatter Plot with annotations",fontsize=15)
# for i, label in enumerate(carrier_stats['AIRLINE']):
#     plt.annotate(label, (carrier_stats['PCT_DELAYED'][i], carrier_stats['AVG_DELAY'][i]))

# plt.show()

In [ ]:
# Get the avg delay for each airport
avg_delays = basecopy.loc[basecopy.DEP_DELAY>=15, ['ORIGIN',
                                                          'DEP_DELAY']].groupby(by='ORIGIN', 
                                                                                as_index=False).mean().rename(columns={'DEP_DELAY':'AVG_DELAY'})


pct_delayed = basecopy[['ORIGIN', 
                                'DEP_DELAY_GE15', 
                                'DEP_DELAY_GE60', 
                                'MONTH']].groupby(by='ORIGIN', 
                                                  as_index=False).agg({'DEP_DELAY_GE15':'sum', 
                                                                       'DEP_DELAY_GE60':'sum',
                                                                       'MONTH':'count'}).rename(columns={'MONTH':'TOTAL_FLIGHTS'})

# print(pct_delayed.sort_values(by=['TOTAL_FLIGHTS']))

# Get the % of flights delays for each airline (as well as number of flights and % of flights delayed >=60)
pct_delayed['PCT_DELAYED']=100*pct_delayed.DEP_DELAY_GE15/pct_delayed.TOTAL_FLIGHTS
pct_delayed['PCT_GE60']=100*pct_delayed.DEP_DELAY_GE60/pct_delayed.TOTAL_FLIGHTS
pct_delayed = pct_delayed[['ORIGIN', 'PCT_DELAYED', 'PCT_GE60', 'TOTAL_FLIGHTS']]


# # Merge the datasets (and bring in the airline name to replace the code)
colm_stats = avg_delays.merge(pct_delayed).sort_values(by=['TOTAL_FLIGHTS'])
colm_stats = avg_delays.merge(pct_delayed)
colm_stats = colm_stats.loc[colm_stats.TOTAL_FLIGHTS>7000].reset_index().drop('index', axis=1)

print(colm_stats)



# Plot the airlines to see which ones had the worst/best delay record
fig, ax = plt.subplots(figsize=(20,12))
ax.scatter(colm_stats['PCT_DELAYED'], colm_stats['AVG_DELAY'], 
           alpha=0.7, marker="p", s=colm_stats['TOTAL_FLIGHTS']/100, color='#029386')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('2019 Delays by Airport', weight='semibold', size=25)
plt.xlabel('% of Flights Delayed', weight='medium', size=20)
plt.ylabel('Avg Delay (mins)', weight='medium', size=20)
for i, label in enumerate(colm_stats['ORIGIN']):
    ax.text(colm_stats['PCT_DELAYED'][i]-0.2, colm_stats['AVG_DELAY'][i]+.8, label,
           weight='semibold', size=15)
ax.text(23, 94, 'Marker size denotes ', size=15, weight='bold', color='#029386')
ax.text(23.2, 92, 'number of flights ', size=15, weight='bold', color='#029386')
plt.show()

In [ ]:
print("VARIABLE: DISTANCE_GROUP")
print("________________")
print("")
print("Value Counts:")
print(training.DISTANCE_GROUP.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.DISTANCE_GROUP, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: The delay % for each group is close to the avg, so no real value here

###  <span style='color:blue'> * Potentially Useful Column Found: TYPE_AIRCRAFT * </span>

In [ ]:
print("VARIABLE: TYPE_AIRCRAFT")
print("________________")
print("")
print("Value Counts:")
print(training.TYPE_AIRCRAFT.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.TYPE_AIRCRAFT, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: There might be some benefit. Fixed wing single engine has lower delay %. 
#             But it might be a proxy for size of airport

###  <span style='color:blue'> * Potentially Useful Column Found: TYPE_ENGINE * </span>

In [ ]:
print("VARIABLE: TYPE_ENGINE")
print("________________")
print("")
print("Value Counts:")
print(training.TYPE_ENGINE.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.TYPE_ENGINE, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: Turbo-prop has a much lower delay %

###  <span style='color:blue'> * Potentially Useful Column Found: NO_ENG * </span>

In [ ]:
print("VARIABLE: NO_ENG")
print("________________")
print("")
print("Value Counts:")
print(training.NO_ENG.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.NO_ENG, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: Might be useful. Single-Engine seems to have a lower delay %

###  <span style='color:blue'> * Potentially Useful Column Found: AC_WEIGHT * </span>

In [ ]:
print("VARIABLE: AC_WEIGHT")
print("________________")
print("")
print("Value Counts:")
print(training.AC_WEIGHT.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.AC_WEIGHT, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

# CONCLUSION: Potentially useful. Look at class 4.

###  <span style='color:blue'> * Potentially Useful Column Found: MFR_Engine * </span>

In [ ]:
print("VARIABLE: MFR_Engine")
print("________________")
print("")
print("Value Counts:")
print(training.MFR_Engine.value_counts())
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.MFR_Engine, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

###  <span style='color:blue'> * Potentially Useful Column Found: MODEL_GROUP_Aircraft * </span>

In [ ]:
print("VARIABLE: MODEL_GROUP_Aircraft")
print("________________")
print("")
print("Value Counts:")
print(training.MODEL_GROUP_Aircraft.value_counts().head(25))
print("")


print("Crosstab Freqs:")
cxtab = pd.crosstab(training.MODEL_GROUP_Aircraft, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

________________________________________________________________________________

# III. Further Preprocessing and Transformations

In [ ]:
#display.Image(os.path.join(a_path, a_folder, "IMG_Transformation.png")) # https://drive.google.com/file/d/1CTYX_kFz59pxKwGESM5LuSeC8B7DRAD2/view?usp=sharing

![picture](https://drive.google.com/uc?id=1CTYX_kFz59pxKwGESM5LuSeC8B7DRAD2)

#### Variable: ORIGIN ----> Becomes: ORIGIN_REDUCED

In [ ]:
# Which origin airports have above avg and below avg number of long delays (avg over all data is 12.2%)
large_ports = training[training.annual_departure_count>130000] # 130000 is the median annual_departure_count
print(large_ports.shape[0])
print("")

cxtab = pd.crosstab(large_ports.ORIGIN, large_ports.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab[cxtab[1]>.122].sort_values(by=1, ascending=False).head(15))

print(cxtab[cxtab[1]<.122].sort_values(by=1, ascending=True).head(15))

In [ ]:
# Make a column that keeps those airports that have above avg long delay rates, and convert all others to "OTHER"
basecopy["ORIGIN_REDUCED"] = basecopy["ORIGIN"]
basecopy.loc[~(basecopy["ORIGIN"].isin(['EWR', 'ORD', 'BOS', 'SFO', 'LGA', 'MCO', 'DEN', 'IAH', 'DFW', 
                                        'SEA', 'ATL', 'MSP'])), 
             "ORIGIN_REDUCED"] = "OTHER"

# Confirm that it worked
# training.loc[0:50, ["ORIGIN", "ORIGIN_REDUCED"]]
print(basecopy.ORIGIN_REDUCED.value_counts())

#### Variable: OP_CARRIER  ---->  Becomes: OP_CARRIER_REDUCED

In [ ]:
print(training.OP_CARRIER.value_counts())

# Which OP_CARRIER have above avg and below avg number of long delays (avg over all data is 12.2%)
cxtab = pd.crosstab(training.OP_CARRIER, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

In [ ]:
# Make a column that keeps those carriers that have above avg long delay rates, and convert all others to "OTHER"
basecopy["OP_CARRIER_REDUCED"] = basecopy["OP_CARRIER"]
basecopy.loc[~(basecopy["OP_CARRIER"].isin(['C5', 'KS', 'AX', 'B6', 'F9', 'EV', 'G7', 'UA', 
                                            'AS', 'EM', 'DL', 'QX', 'HA', 
                                            'WN', 'DL', 'AA', '00', 'YX', 'MQ', 'OH', '9E', 'YV', 'NK'])), 
             "OP_CARRIER_REDUCED"] = "OTHER"

# Confirm that it worked
# training.loc[0:50, ["OP_CARRIER", "OP_CARRIER_REDUCED"]]
print(basecopy.OP_CARRIER_REDUCED.value_counts())

#### Variable: ACFT_MFR_DeDup ----> Becomes: ACFT_MFR_REDUCED

In [ ]:
basecopy.ACFT_MFR_DeDup.value_counts()

In [ ]:
# Make a column that keeps most frequent Aircraft Manufacturers and groups all low-count mfctrs as "OTHER"
basecopy["ACFT_MFR_REDUCED"] = basecopy["ACFT_MFR_DeDup"]

basecopy.loc[pd.isna(basecopy["ACFT_MFR_REDUCED"]), "ACFT_MFR_REDUCED"] = "UNKNOWN"

basecopy.loc[~(basecopy["ACFT_MFR_REDUCED"].isin(['UNKNOWN', 'BOEING', 'AIRBUS', 'EMBRAER', 'BOMBARDIER INC', 
                                                'MCDONNELL DOUGLAS', 'C SERIES AIRCRAFT LTD PTNRSP', 'ALENIA',
                                                 'PIPER', 'BELL HELICOPTER TEXTRON CANADA', 'CANADAIR', 
                                                  'AMERICAN CHAMPION AIRCRAFT', 'SAAB'])), 
             "ACFT_MFR_REDUCED"] = "OTHER"

basecopy.loc[basecopy.ACFT_MFR_REDUCED=="BOMBARDIER INC", "ACFT_MFR_REDUCED"] = "BOMBARDIER"
basecopy.loc[basecopy.ACFT_MFR_REDUCED=="MCDONNELL DOUGLAS", "ACFT_MFR_REDUCED"] = "MCDONNELL"
basecopy.loc[basecopy.ACFT_MFR_REDUCED=="C SERIES AIRCRAFT LTD PTNRSP", "ACFT_MFR_REDUCED"] = "CSERIES"
basecopy.loc[basecopy.ACFT_MFR_REDUCED=="BELL HELICOPTER TEXTRON CANADA", "ACFT_MFR_REDUCED"] = "BELL"
basecopy.loc[basecopy.ACFT_MFR_REDUCED=="AMERICAN CHAMPION AIRCRAFT", "ACFT_MFR_REDUCED"] = "CHAMPION"

# Confirm that it worked
print(basecopy.ACFT_MFR_REDUCED.value_counts())
# print(training.loc[~(training["ACFT_MFR_DeDup"].isin(['BOEING', 'AIRBUS', 'EMBRAER', 'BOMBARDIER INC', 
#                                                 'MCDONNELL DOUGLAS', 'C SERIES AIRCRAFT LTD PTNRSP', 'ALENIA'])), 
#              "ACFT_MFR_REDUCED"].shape)
# print(training[training.ACFT_MFR_REDUCED=='OTHER'].shape)
# training.loc[0:50, ["ACFT_MFR_DeDup", "ACFT_MFR_REDUCED"]]

In [ ]:
# Get the avg delay for each ACFT_MFR_REDUCED
avg_delays = basecopy.loc[basecopy.DEP_DELAY>=15, ['ACFT_MFR_REDUCED',
                                                          'DEP_DELAY']].groupby(by='ACFT_MFR_REDUCED', 
                                                                                as_index=False).mean().rename(columns={'DEP_DELAY':'AVG_DELAY'})


pct_delayed = basecopy[['ACFT_MFR_REDUCED', 
                                'DEP_DELAY_GE15', 
                                'DEP_DELAY_GE60', 
                                'MONTH']].groupby(by='ACFT_MFR_REDUCED', 
                                                  as_index=False).agg({'DEP_DELAY_GE15':'sum', 
                                                                       'DEP_DELAY_GE60':'sum',
                                                                       'MONTH':'count'}).rename(columns={'MONTH':'TOTAL_FLIGHTS'})

# print(pct_delayed.sort_values(by=['TOTAL_FLIGHTS']))

# Get the % of flights delays for each airline (as well as number of flights and % of flights delayed >=60)
pct_delayed['PCT_DELAYED']=100*pct_delayed.DEP_DELAY_GE15/pct_delayed.TOTAL_FLIGHTS
pct_delayed['PCT_GE60']=100*pct_delayed.DEP_DELAY_GE60/pct_delayed.TOTAL_FLIGHTS
pct_delayed = pct_delayed[['ACFT_MFR_REDUCED', 'PCT_DELAYED', 'PCT_GE60', 'TOTAL_FLIGHTS']]


# # Merge the datasets (and bring in the airline name to replace the code)
colm_stats = avg_delays.merge(pct_delayed).sort_values(by=['TOTAL_FLIGHTS'])
colm_stats = avg_delays.merge(pct_delayed)
# colm_stats = colm_stats.loc[colm_stats.TOTAL_FLIGHTS>7000].reset_index().drop('index', axis=1)

print(colm_stats)



# Plot the airlines to see which ones had the worst/best delay record
fig, ax = plt.subplots(figsize=(20,12))
ax.scatter(colm_stats['PCT_DELAYED'], colm_stats['AVG_DELAY'], 
           alpha=0.7, marker=",", s=colm_stats['TOTAL_FLIGHTS']/200)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('2019 Delays by Aircraft Manufacturer', weight='semibold', size=25)
plt.xlabel('% of Flights Delayed', weight='medium', size=20)
plt.ylabel('Avg Delay (mins)', weight='medium', size=20)
for i, label in enumerate(colm_stats['ACFT_MFR_REDUCED']):
    ax.text(colm_stats['PCT_DELAYED'][i]-0.2, colm_stats['AVG_DELAY'][i]+.8, label,
           weight='semibold', size=15)
ax.text(8.7, 96, 'Square size denotes ', size=15, weight='bold', color='#1f77b4')
ax.text(8.9, 94, 'number of flights ', size=15, weight='bold', color='#1f77b4')
plt.show()

#### Variable: MFR_Engine ----> Becomes: MFR_ENGINE_REDUCED

In [ ]:
print("MFR_Engine    Count")
print(training.MFR_Engine.value_counts())

In [ ]:
# Make a column that keeps most frequent Engine Manufacturers and groups all low-count mfctrs as "OTHER"
basecopy["MFR_ENGINE_REDUCED"] = basecopy["MFR_Engine"]

basecopy.loc[pd.isna(basecopy["MFR_ENGINE_REDUCED"]), "MFR_ENGINE_REDUCED"] = "UNKNOWN"

basecopy.loc[basecopy.MFR_ENGINE_REDUCED=="CFM INTL", "MFR_ENGINE_REDUCED"] = "CFM"
basecopy.loc[basecopy.MFR_ENGINE_REDUCED=="ROLLS-ROYC", "MFR_ENGINE_REDUCED"] = "ROLLS"
basecopy.loc[basecopy.MFR_ENGINE_REDUCED=="P&W", "MFR_ENGINE_REDUCED"] = "PW"
basecopy.loc[basecopy.MFR_ENGINE_REDUCED=="AMA/EXPR", "MFR_ENGINE_REDUCED"] = "AMA"

basecopy.loc[~(basecopy["MFR_ENGINE_REDUCED"].isin(['UNKNOWN', 'CFM', 'GE', 'IAE', 'ROLLS', 'PW', 
                                                    'ALLISON', 'AMA', 'LYCOMING'])), 
             "MFR_ENGINE_REDUCED"] = "OTHER"

# Confirm that it worked
print(basecopy.MFR_ENGINE_REDUCED.value_counts())

In [ ]:
# Get the avg delay for each engine manufacturer
avg_delays = basecopy.loc[basecopy.DEP_DELAY>=15, ['MFR_ENGINE_REDUCED',
                                                          'DEP_DELAY']].groupby(by='MFR_ENGINE_REDUCED', 
                                                                                as_index=False).mean().rename(columns={'DEP_DELAY':'AVG_DELAY'})


pct_delayed = basecopy[['MFR_ENGINE_REDUCED', 
                                'DEP_DELAY_GE15', 
                                'DEP_DELAY_GE60', 
                                'MONTH']].groupby(by='MFR_ENGINE_REDUCED', 
                                                  as_index=False).agg({'DEP_DELAY_GE15':'sum', 
                                                                       'DEP_DELAY_GE60':'sum',
                                                                       'MONTH':'count'}).rename(columns={'MONTH':'TOTAL_FLIGHTS'})

# print(pct_delayed.sort_values(by=['TOTAL_FLIGHTS']))

# Get the % of flights delays for each airline (as well as number of flights and % of flights delayed >=60)
pct_delayed['PCT_DELAYED']=100*pct_delayed.DEP_DELAY_GE15/pct_delayed.TOTAL_FLIGHTS
pct_delayed['PCT_GE60']=100*pct_delayed.DEP_DELAY_GE60/pct_delayed.TOTAL_FLIGHTS
pct_delayed = pct_delayed[['MFR_ENGINE_REDUCED', 'PCT_DELAYED', 'PCT_GE60', 'TOTAL_FLIGHTS']]


# # Merge the datasets (and bring in the airline name to replace the code)
colm_stats = avg_delays.merge(pct_delayed).sort_values(by=['TOTAL_FLIGHTS'])
colm_stats = avg_delays.merge(pct_delayed)
# colm_stats = colm_stats.loc[colm_stats.TOTAL_FLIGHTS>7000].reset_index().drop('index', axis=1)

print(colm_stats)



# Plot the airlines to see which ones had the worst/best delay record
fig, ax = plt.subplots(figsize=(20,12))
ax.scatter(colm_stats['PCT_DELAYED'], colm_stats['AVG_DELAY'], 
           alpha=0.7, marker=",", s=colm_stats['TOTAL_FLIGHTS']/200)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('2019 Delays by Engine Manufacturer', weight='semibold', size=25)
plt.xlabel('% of Flights Delayed', weight='medium', size=20)
plt.ylabel('Avg Delay (mins)', weight='medium', size=20)
for i, label in enumerate(colm_stats['MFR_ENGINE_REDUCED']):
    ax.text(colm_stats['PCT_DELAYED'][i]-0.2, colm_stats['AVG_DELAY'][i]+.8, label,
           weight='semibold', size=15)
ax.text(8.7, 96, 'Square size denotes ', size=15, weight='bold', color='#1f77b4')
ax.text(8.9, 94, 'number of flights ', size=15, weight='bold', color='#1f77b4')
plt.show()

#### Variable: CRS_DEP_TIME ----> Becomes: CRS_DEP_TIME_BUCKET

In [ ]:
basecopy.loc[(basecopy.CRS_DEP_TIME>=500) & (basecopy.CRS_DEP_TIME<900), "CRS_DEP_TIME_BUCKET"] = "5am-9am"
basecopy.loc[(basecopy.CRS_DEP_TIME>=900) & (basecopy.CRS_DEP_TIME<1300), "CRS_DEP_TIME_BUCKET"] = "9am-1pm"
basecopy.loc[(basecopy.CRS_DEP_TIME>=1300) & (basecopy.CRS_DEP_TIME<1700), "CRS_DEP_TIME_BUCKET"] = "1pm-5pm"
basecopy.loc[(basecopy.CRS_DEP_TIME>=1700) & (basecopy.CRS_DEP_TIME<2100), "CRS_DEP_TIME_BUCKET"] = "5pm-9pm"
basecopy.loc[~((basecopy.CRS_DEP_TIME>=500) & (basecopy.CRS_DEP_TIME<2100)), "CRS_DEP_TIME_BUCKET"] = "Redeye"

# Confirm that it worked
print(basecopy.CRS_DEP_TIME_BUCKET.value_counts())
# basecopy.loc[0:15, ["CRS_DEP_TIME", "CRS_DEP_TIME_BUCKET"]]
print("")
cxtab = pd.crosstab(basecopy.CRS_DEP_TIME_BUCKET, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')
print(cxtab.sort_values(by=1, ascending=False))

In [ ]:
# Get the avg delay for each time bucket
avg_delays = basecopy.loc[basecopy.DEP_DELAY>=15, ['CRS_DEP_TIME_BUCKET',
                                                          'DEP_DELAY']].groupby(by='CRS_DEP_TIME_BUCKET', 
                                                                                as_index=False).mean().rename(columns={'DEP_DELAY':'AVG_DELAY'})


pct_delayed = basecopy[['CRS_DEP_TIME_BUCKET', 
                                'DEP_DELAY_GE15', 
                                'DEP_DELAY_GE60', 
                                'MONTH']].groupby(by='CRS_DEP_TIME_BUCKET', 
                                                  as_index=False).agg({'DEP_DELAY_GE15':'sum', 
                                                                       'DEP_DELAY_GE60':'sum',
                                                                       'MONTH':'count'}).rename(columns={'MONTH':'TOTAL_FLIGHTS'})

# print(pct_delayed.sort_values(by=['TOTAL_FLIGHTS']))

# Get the % of flights delays for each airline (as well as number of flights and % of flights delayed >=60)
pct_delayed['PCT_DELAYED']=100*pct_delayed.DEP_DELAY_GE15/pct_delayed.TOTAL_FLIGHTS
pct_delayed['PCT_GE60']=100*pct_delayed.DEP_DELAY_GE60/pct_delayed.TOTAL_FLIGHTS
pct_delayed = pct_delayed[['CRS_DEP_TIME_BUCKET', 'PCT_DELAYED', 'PCT_GE60', 'TOTAL_FLIGHTS']]


# # Merge the datasets (and bring in the airline name to replace the code)
colm_stats = avg_delays.merge(pct_delayed).sort_values(by=['TOTAL_FLIGHTS'])
colm_stats = avg_delays.merge(pct_delayed)
# colm_stats = colm_stats.loc[colm_stats.TOTAL_FLIGHTS>7000].reset_index().drop('index', axis=1)

print(colm_stats)



# Plot the airlines to see which ones had the worst/best delay record
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(colm_stats['PCT_DELAYED'], colm_stats['AVG_DELAY'], 
           alpha=0.7, marker="o", s=colm_stats['TOTAL_FLIGHTS']/1000, color="#5D3A9B")
ax.set_ylim([60, 90])
ax.set_xlim([5, 30])
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('Delays by Scheduled Departure Time', weight='semibold', size=25)
plt.xlabel('% of Flights Delayed', weight='medium', size=20)
plt.ylabel('Avg Delay (mins)', weight='medium', size=20)
for i, label in enumerate(colm_stats['CRS_DEP_TIME_BUCKET']):
    ax.text(colm_stats['PCT_DELAYED'][i]-1.3, colm_stats['AVG_DELAY'][i]+1, label,
           weight='semibold', size=15)
ax.text(20, 87, 'Circle size denotes ', size=15, weight='bold', color='#5D3A9B')
ax.text(20.3, 85, 'number of flights ', size=15, weight='bold', color='#5D3A9B')
plt.show()

#### Variable: YEAR_MFR ----> Becomes: ACFT_AGE

In [ ]:
basecopy["ACFT_AGE"] = 2019 - basecopy.YEAR_MFR

# Confirm it worked
basecopy.loc[0:5, ["YEAR_MFR", "ACFT_AGE"]]

#### Variable: MODEL_GROUP_Aircraft ----> Becomes: ACFT_MODEL_REDUCED

In [ ]:
print("Model    Count")
print(training.MODEL_GROUP_Aircraft.value_counts().head(25))
print("")

In [ ]:
basecopy["ACFT_MODEL_REDUCED"] = basecopy["MODEL_GROUP_Aircraft"]

basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="CL-600", "ACFT_MODEL_REDUCED"] = "CL600"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="ERJ 170", "ACFT_MODEL_REDUCED"] = "ERJ170"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="EMB-145", "ACFT_MODEL_REDUCED"] = "EMB145"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="717-200", "ACFT_MODEL_REDUCED"] = "717_200"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="ERJ 190", "ACFT_MODEL_REDUCED"] = "ERJ190"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="MD-88", "ACFT_MODEL_REDUCED"] = "MD88"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="EMB-135KL", "ACFT_MODEL_REDUCED"] = "EMB135KL"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="DHC-8", "ACFT_MODEL_REDUCED"] = "DHC8"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="MD-90-30", "ACFT_MODEL_REDUCED"] = "MD90_30"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="DC-9", "ACFT_MODEL_REDUCED"] = "DC9"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="BD-500-1A10", "ACFT_MODEL_REDUCED"] = "BD500"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED.astype(str).str.contains('A330', case=False), "ACFT_MODEL_REDUCED"] = "A330"
basecopy.loc[basecopy.ACFT_MODEL_REDUCED=="ATR42-500", "ACFT_MODEL_REDUCED"] = "ATR42_500"

basecopy.loc[~(basecopy["ACFT_MODEL_REDUCED"].isin(['nan', '737', 'CL600', 'ERJ170', 'A320', 'A321', 'EMB145', 'A319',
                                                   '717_200', '757', 'ERJ190', 'MD88', 'EMB135KL', 'DHC8', 'MD90_30',
                                                   'DC9', '777', '767', 'BD500', 'A330', 'ATR42_500', '787'])), 
             "ACFT_MODEL_REDUCED"] = "OTHER"


# Confirm it work and look at crosstab freq table
print("Model    Count")
print(basecopy.ACFT_MODEL_REDUCED.value_counts())
print("")

print("Crosstab Freqs:")
cxtab = pd.crosstab(basecopy.ACFT_MODEL_REDUCED, training.DEP_DELAY_GE30, 
                               margins=True, margins_name="Total", normalize='index')

print(cxtab.sort_values(by=1, ascending=False))

In [ ]:
# Get the avg delay for each engine manufacturer
avg_delays = basecopy.loc[basecopy.DEP_DELAY>=15, ['ACFT_MODEL_REDUCED',
                                                          'DEP_DELAY']].groupby(by='ACFT_MODEL_REDUCED', 
                                                                                as_index=False).mean().rename(columns={'DEP_DELAY':'AVG_DELAY'})


pct_delayed = basecopy[['ACFT_MODEL_REDUCED', 
                                'DEP_DELAY_GE15', 
                                'DEP_DELAY_GE60', 
                                'MONTH']].groupby(by='ACFT_MODEL_REDUCED', 
                                                  as_index=False).agg({'DEP_DELAY_GE15':'sum', 
                                                                       'DEP_DELAY_GE60':'sum',
                                                                       'MONTH':'count'}).rename(columns={'MONTH':'TOTAL_FLIGHTS'})

# print(pct_delayed.sort_values(by=['TOTAL_FLIGHTS']))

# Get the % of flights delays for each airline (as well as number of flights and % of flights delayed >=60)
pct_delayed['PCT_DELAYED']=100*pct_delayed.DEP_DELAY_GE15/pct_delayed.TOTAL_FLIGHTS
pct_delayed['PCT_GE60']=100*pct_delayed.DEP_DELAY_GE60/pct_delayed.TOTAL_FLIGHTS
pct_delayed = pct_delayed[['ACFT_MODEL_REDUCED', 'PCT_DELAYED', 'PCT_GE60', 'TOTAL_FLIGHTS']]


# Merge the datasets (and bring in the airline name to replace the code)
colm_stats = avg_delays.merge(pct_delayed).sort_values(by=['TOTAL_FLIGHTS'])
colm_stats = avg_delays.merge(pct_delayed)
colm_stats = colm_stats.loc[colm_stats.ACFT_MODEL_REDUCED != 'nan'].reset_index().drop('index', axis=1)
colm_stats = colm_stats.loc[colm_stats.ACFT_MODEL_REDUCED != 'OTHER'].reset_index().drop('index', axis=1)

print(colm_stats)



# Plot the airlines to see which ones had the worst/best delay record
fig, ax = plt.subplots(figsize=(20,12))
ax.scatter(colm_stats['PCT_DELAYED'], colm_stats['AVG_DELAY'], 
           alpha=0.7, marker=">", s=colm_stats['TOTAL_FLIGHTS']/150, color='#00B9F1')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title('2019 Delays by Aircraft Model', weight='semibold', size=25)
plt.xlabel('% of Flights Delayed', weight='medium', size=20)
plt.ylabel('Avg Delay (mins)', weight='medium', size=20)
for i, label in enumerate(colm_stats['ACFT_MODEL_REDUCED']):
    ax.text(colm_stats['PCT_DELAYED'][i]-0.15, colm_stats['AVG_DELAY'][i]+.8, label,
           weight='semibold', size=15)
ax.text(10.7, 95, 'Triangle size denotes ', size=15, weight='bold', color='#00B9F1')
ax.text(10.9, 93, 'number of flights ', size=15, weight='bold', color='#00B9F1')
plt.show()

### Log and Polynomial Transforms

In [ ]:
basecopy["LOG_DISTANCE"]= np.log(basecopy["DISTANCE"])
basecopy[["DISTANCE", "LOG_DISTANCE"]].head(3)

In [ ]:
basecopy["LOG_FLIGHTS_BTWN"]= np.log(basecopy["annual_number_of_flights_btwn"])
basecopy[["annual_number_of_flights_btwn", "LOG_FLIGHTS_BTWN"]].head(3)

In [ ]:
basecopy["LOG_DEPT_COUNT"]= np.log(basecopy["annual_departure_count"])
basecopy[["annual_departure_count", "LOG_DEPT_COUNT"]].head(3)

In [ ]:
basecopy["LOG_NO_SEATS"]= np.log(basecopy["NO_SEATS"]+1)
basecopy[["NO_SEATS", "LOG_NO_SEATS"]].head(3)

In [ ]:
print(basecopy[basecopy.ENGREF_MATCHES==True].shape)
print(basecopy[~(basecopy.ENGREF_MATCHES==True)].shape)
print(basecopy[basecopy.ENGREF_MATCHES==False].shape)
print(basecopy[pd.isnull(basecopy.ENGREF_MATCHES)].shape)

### Remove unneeded columns

"basekeep" will be the dataset we move forward with for model development

In [ ]:
print(basecopy.shape)
basekeep = basecopy.copy()

# Only keep rows where the engine info from the ENGINE table matches what is in the aircraft table.
basekeep = basekeep[basekeep.ENGREF_MATCHES==True]
print(basekeep.shape)

# This will be the dataset we move forward with for model development
basekeep = basekeep[["DEP_DELAY", "DEP_DELAY_HR", "DEP_DELAY_GE15","DEP_DELAY_GE30", "DEP_DELAY_GE45", "DEP_DELAY_GE60",
                     "MONTH", "DAY_OF_WEEK", "OP_CARRIER_REDUCED", "ORIGIN_REDUCED", "CRS_DEP_TIME_BUCKET", 
                     "DISTANCE", "LOG_DISTANCE", "ACFT_AGE", "TYPE_AIRCRAFT", "TYPE_ENGINE", "NO_ENG", 
                     "NO_SEATS", "LOG_NO_SEATS", "AC_WEIGHT", "MFR_ENGINE_REDUCED", "THRUST", 
                     "ACFT_MFR_REDUCED", "annual_departure_count", "LOG_DEPT_COUNT", "airline_count", 
                     "annual_number_of_flights_btwn", "LOG_FLIGHTS_BTWN", "ACTIVE_AIRCRAFT_CNT_2018", 
                     "ANNUAL_FLT_CNT_2018", "ANNUAL_AIR_MILES_2018", "ACFT_MODEL_REDUCED"]]

print(basekeep.shape)
print("")
print("Missing Value counts: ")
print(basekeep.isnull().sum()[basekeep.isnull().sum()>0])

### Imputation and/or Removal of null values

In [ ]:
# Looking at the data, when DEP_DELAY is missing, the flight was cancelled. So remove these records.
basekeep = basekeep[~(basekeep.DEP_DELAY.isnull())]

# Also remove records where missing annual_number_of_flights_btwn
basekeep = basekeep[~(basekeep.annual_number_of_flights_btwn.isnull())]

print(basekeep.shape)
print("")
print("Missing Value counts: ")
print(basekeep.isnull().sum()[basekeep.isnull().sum()>0])

In [ ]:
# Impute missing aircraft age with the median age pre aircraft model

# First create a dataframe of median age for each acft model
medianages = pd.DataFrame(basekeep.groupby("ACFT_MODEL_REDUCED")["ACFT_AGE"].median())
medianages["Median_Age"] = medianages.ACFT_AGE
medianages.reset_index(inplace=True)
medianages= medianages[["ACFT_MODEL_REDUCED", "Median_Age"]]
print(medianages)

# Join that dataframe and replace missing ACFT_AGE values. The new imputed column is called ACFT_AGE_IMPUTED
basekeep = basekeep.merge(medianages, on='ACFT_MODEL_REDUCED', how='left')
basekeep["ACFT_AGE_IMPUTED"] = basekeep["ACFT_AGE"]
basekeep.loc[basekeep["ACFT_AGE_IMPUTED"].isnull(),"ACFT_AGE_IMPUTED"]=basekeep["Median_Age"]

basekeep.drop("Median_Age", axis=1, inplace=True)

In [ ]:
print(basekeep.shape) 
print(basekeep[["ACFT_AGE", "ACFT_AGE_IMPUTED"]])

basekeep.drop("ACFT_AGE", axis=1, inplace=True)
print(basekeep.shape) 

In [ ]:
# Create a polynomial feature for Age variable
basekeep["ACFT_AGE_IMPUTED_2"] = basekeep["ACFT_AGE_IMPUTED"]**2
basekeep[["ACFT_AGE_IMPUTED", "ACFT_AGE_IMPUTED_2"]].head(3)

### Create dummy columns for categorical variables

In [ ]:
# Create dummies for categorical columns
base_dummied = pd.get_dummies(basekeep, 
                           columns=['MONTH', 'DAY_OF_WEEK', "OP_CARRIER_REDUCED", 
                                    "ORIGIN_REDUCED", "CRS_DEP_TIME_BUCKET", "TYPE_AIRCRAFT", 
                                    "TYPE_ENGINE", "NO_ENG", "AC_WEIGHT", "MFR_ENGINE_REDUCED", 
                                    "ACFT_MFR_REDUCED", "ACFT_MODEL_REDUCED"])

print(base_dummied.shape)
print(base_dummied.columns[0:100])
print(base_dummied.columns[100:119])

In [ ]:
base_dummied.shape

In [ ]:
len(base_dummied.DEP_DELAY)

# IV. Model Building

In [ ]:
#display.Image(os.path.join(a_path, a_folder, "IMG_Construction.jpg")) # https://drive.google.com/file/d/1GgyuK9MjBWR-hNlLleEeRfK-w6876k5S/view?usp=sharing

![picture](https://drive.google.com/uc?id=1GgyuK9MjBWR-hNlLleEeRfK-w6876k5S)

# PCA

________________________________________________________________________________

Find the Principle components to reduce the dimensionality
[Wicked animated GIF which illustrates PCA](http://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues)

Magically, eigen-decomposition (or PCA) finds the line where
1. the spread of values along the black line is **maximal**
2. the projection error (sum of red lines) is **minimal**

<img src="https://i.stack.imgur.com/Q7HIP.gif">

In [ ]:
base_dummied.isna().sum()

In [ ]:
[ i for i in base_dummied.columns]

In [ ]:
base_dummied_final=base_dummied.drop(labels=['DEP_DELAY','DEP_DELAY_GE15','DEP_DELAY_GE30','DEP_DELAY_GE45','DEP_DELAY_GE60','DEP_DELAY_HR'], axis=1)

In [ ]:
print(base_dummied.shape)
print(base_dummied_final.shape)

In [ ]:
print(len(base_dummied))
print(len(base_dummied_final))

In [ ]:
base_dummied_regression_Y=base_dummied['DEP_DELAY']
base_dummied_regression_Y.shape

In [ ]:
len([i for i in base_dummied.DEP_DELAY])

In [ ]:
len(base_dummied['DEP_DELAY'])

In [ ]:
base_dummied_final.describe()

In [ ]:
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
base_dummied_final.head()

In [ ]:
scaler_df=base_dummied_final[[
 'DISTANCE',
 'LOG_DISTANCE',
 'NO_SEATS',
 'LOG_NO_SEATS',
 'THRUST',
 'annual_departure_count',
 'LOG_DEPT_COUNT',
 'airline_count',
 'annual_number_of_flights_btwn',
 'LOG_FLIGHTS_BTWN',
 'ACTIVE_AIRCRAFT_CNT_2018',
 'ANNUAL_FLT_CNT_2018',
 'ANNUAL_AIR_MILES_2018',
 'ACFT_AGE_IMPUTED',
 'ACFT_AGE_IMPUTED_2']
 ]

In [ ]:
scaler_df.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_data=scaler.fit_transform(scaler_df)

In [ ]:
scaled_data_final=pd.DataFrame(data=scaled_data, columns= scaler_df.columns)

In [ ]:
scaled_data_final.describe()

In [ ]:
scaled_data_final.shape

In [ ]:
base_dummied_final.shape

In [ ]:
base_dummied_final.head()

In [ ]:
scaled_data_final[[
 'MONTH_1',
 'MONTH_2',
 'MONTH_3',
 'MONTH_4',
 'MONTH_5',
 'MONTH_6',
 'MONTH_7',
 'MONTH_8',
 'MONTH_9',
 'MONTH_10',
 'MONTH_11',
 'MONTH_12',
 'DAY_OF_WEEK_1',
 'DAY_OF_WEEK_2',
 'DAY_OF_WEEK_3',
 'DAY_OF_WEEK_4',
 'DAY_OF_WEEK_5',
 'DAY_OF_WEEK_6',
 'DAY_OF_WEEK_7',
 'OP_CARRIER_REDUCED_9E',
 'OP_CARRIER_REDUCED_AA',
 'OP_CARRIER_REDUCED_AS',
 'OP_CARRIER_REDUCED_AX',
 'OP_CARRIER_REDUCED_B6',
 'OP_CARRIER_REDUCED_C5',
 'OP_CARRIER_REDUCED_DL',
 'OP_CARRIER_REDUCED_EM',
 'OP_CARRIER_REDUCED_EV',
 'OP_CARRIER_REDUCED_F9',
 'OP_CARRIER_REDUCED_G7',
 'OP_CARRIER_REDUCED_HA',
 'OP_CARRIER_REDUCED_KS',
 'OP_CARRIER_REDUCED_MQ',
 'OP_CARRIER_REDUCED_NK',
 'OP_CARRIER_REDUCED_OH',
 'OP_CARRIER_REDUCED_OTHER',
 'OP_CARRIER_REDUCED_QX',
 'OP_CARRIER_REDUCED_UA',
 'OP_CARRIER_REDUCED_WN',
 'OP_CARRIER_REDUCED_YV',
 'OP_CARRIER_REDUCED_YX',
 'ORIGIN_REDUCED_ATL',
 'ORIGIN_REDUCED_BOS',
 'ORIGIN_REDUCED_DEN',
 'ORIGIN_REDUCED_DFW',
 'ORIGIN_REDUCED_EWR',
 'ORIGIN_REDUCED_IAH',
 'ORIGIN_REDUCED_LGA',
 'ORIGIN_REDUCED_MCO',
 'ORIGIN_REDUCED_MSP',
 'ORIGIN_REDUCED_ORD',
 'ORIGIN_REDUCED_OTHER',
 'ORIGIN_REDUCED_SEA',
 'ORIGIN_REDUCED_SFO',
 'CRS_DEP_TIME_BUCKET_1pm-5pm',
 'CRS_DEP_TIME_BUCKET_5am-9am',
 'CRS_DEP_TIME_BUCKET_5pm-9pm',
 'CRS_DEP_TIME_BUCKET_9am-1pm',
 'CRS_DEP_TIME_BUCKET_Redeye',
 'TYPE_AIRCRAFT_Fixed wing multi engine',
 'TYPE_AIRCRAFT_Fixed wing single engine',
 'TYPE_AIRCRAFT_Rotorcraft',
 'TYPE_ENGINE_Reciprocating',
 'TYPE_ENGINE_Turbo-fan',
 'TYPE_ENGINE_Turbo-jet',
 'TYPE_ENGINE_Turbo-prop',
 'TYPE_ENGINE_Turbo-shaft',
 'NO_ENG_1.0',
 'NO_ENG_2.0',
 'NO_ENG_3.0',
 'AC_WEIGHT_CLASS 1',
 'AC_WEIGHT_CLASS 2',
 'AC_WEIGHT_CLASS 3',
 'MFR_ENGINE_REDUCED_ALLISON',
 'MFR_ENGINE_REDUCED_CFM',
 'MFR_ENGINE_REDUCED_GE',
 'MFR_ENGINE_REDUCED_IAE',
 'MFR_ENGINE_REDUCED_LYCOMING',
 'MFR_ENGINE_REDUCED_OTHER',
 'MFR_ENGINE_REDUCED_PW',
 'MFR_ENGINE_REDUCED_ROLLS',
 'ACFT_MFR_REDUCED_AIRBUS',
 'ACFT_MFR_REDUCED_ALENIA',
 'ACFT_MFR_REDUCED_BELL',
 'ACFT_MFR_REDUCED_BOEING',
 'ACFT_MFR_REDUCED_BOMBARDIER',
 'ACFT_MFR_REDUCED_CANADAIR',
 'ACFT_MFR_REDUCED_CHAMPION',
 'ACFT_MFR_REDUCED_CSERIES',
 'ACFT_MFR_REDUCED_EMBRAER',
 'ACFT_MFR_REDUCED_MCDONNELL',
 'ACFT_MFR_REDUCED_OTHER',
 'ACFT_MFR_REDUCED_PIPER',
 'ACFT_MFR_REDUCED_SAAB',
 'ACFT_MODEL_REDUCED_717_200',
 'ACFT_MODEL_REDUCED_737',
 'ACFT_MODEL_REDUCED_757',
 'ACFT_MODEL_REDUCED_767',
 'ACFT_MODEL_REDUCED_777',
 'ACFT_MODEL_REDUCED_787',
 'ACFT_MODEL_REDUCED_A319',
 'ACFT_MODEL_REDUCED_A320',
 'ACFT_MODEL_REDUCED_A321',
 'ACFT_MODEL_REDUCED_A330',
 'ACFT_MODEL_REDUCED_ATR42_500',
 'ACFT_MODEL_REDUCED_BD500',
 'ACFT_MODEL_REDUCED_CL600',
 'ACFT_MODEL_REDUCED_DHC8',
 'ACFT_MODEL_REDUCED_EMB135KL',
 'ACFT_MODEL_REDUCED_EMB145',
 'ACFT_MODEL_REDUCED_ERJ170',
 'ACFT_MODEL_REDUCED_ERJ190',
 'ACFT_MODEL_REDUCED_MD88',
 'ACFT_MODEL_REDUCED_MD90_30',
 'ACFT_MODEL_REDUCED_OTHER']]=base_dummied_final[[
 'MONTH_1',
 'MONTH_2',
 'MONTH_3',
 'MONTH_4',
 'MONTH_5',
 'MONTH_6',
 'MONTH_7',
 'MONTH_8',
 'MONTH_9',
 'MONTH_10',
 'MONTH_11',
 'MONTH_12',
 'DAY_OF_WEEK_1',
 'DAY_OF_WEEK_2',
 'DAY_OF_WEEK_3',
 'DAY_OF_WEEK_4',
 'DAY_OF_WEEK_5',
 'DAY_OF_WEEK_6',
 'DAY_OF_WEEK_7',
 'OP_CARRIER_REDUCED_9E',
 'OP_CARRIER_REDUCED_AA',
 'OP_CARRIER_REDUCED_AS',
 'OP_CARRIER_REDUCED_AX',
 'OP_CARRIER_REDUCED_B6',
 'OP_CARRIER_REDUCED_C5',
 'OP_CARRIER_REDUCED_DL',
 'OP_CARRIER_REDUCED_EM',
 'OP_CARRIER_REDUCED_EV',
 'OP_CARRIER_REDUCED_F9',
 'OP_CARRIER_REDUCED_G7',
 'OP_CARRIER_REDUCED_HA',
 'OP_CARRIER_REDUCED_KS',
 'OP_CARRIER_REDUCED_MQ',
 'OP_CARRIER_REDUCED_NK',
 'OP_CARRIER_REDUCED_OH',
 'OP_CARRIER_REDUCED_OTHER',
 'OP_CARRIER_REDUCED_QX',
 'OP_CARRIER_REDUCED_UA',
 'OP_CARRIER_REDUCED_WN',
 'OP_CARRIER_REDUCED_YV',
 'OP_CARRIER_REDUCED_YX',
 'ORIGIN_REDUCED_ATL',
 'ORIGIN_REDUCED_BOS',
 'ORIGIN_REDUCED_DEN',
 'ORIGIN_REDUCED_DFW',
 'ORIGIN_REDUCED_EWR',
 'ORIGIN_REDUCED_IAH',
 'ORIGIN_REDUCED_LGA',
 'ORIGIN_REDUCED_MCO',
 'ORIGIN_REDUCED_MSP',
 'ORIGIN_REDUCED_ORD',
 'ORIGIN_REDUCED_OTHER',
 'ORIGIN_REDUCED_SEA',
 'ORIGIN_REDUCED_SFO',
 'CRS_DEP_TIME_BUCKET_1pm-5pm',
 'CRS_DEP_TIME_BUCKET_5am-9am',
 'CRS_DEP_TIME_BUCKET_5pm-9pm',
 'CRS_DEP_TIME_BUCKET_9am-1pm',
 'CRS_DEP_TIME_BUCKET_Redeye',
 'TYPE_AIRCRAFT_Fixed wing multi engine',
 'TYPE_AIRCRAFT_Fixed wing single engine',
 'TYPE_AIRCRAFT_Rotorcraft',
 'TYPE_ENGINE_Reciprocating',
 'TYPE_ENGINE_Turbo-fan',
 'TYPE_ENGINE_Turbo-jet',
 'TYPE_ENGINE_Turbo-prop',
 'TYPE_ENGINE_Turbo-shaft',
 'NO_ENG_1.0',
 'NO_ENG_2.0',
 'NO_ENG_3.0',
 'AC_WEIGHT_CLASS 1',
 'AC_WEIGHT_CLASS 2',
 'AC_WEIGHT_CLASS 3',
 'MFR_ENGINE_REDUCED_ALLISON',
 'MFR_ENGINE_REDUCED_CFM',
 'MFR_ENGINE_REDUCED_GE',
 'MFR_ENGINE_REDUCED_IAE',
 'MFR_ENGINE_REDUCED_LYCOMING',
 'MFR_ENGINE_REDUCED_OTHER',
 'MFR_ENGINE_REDUCED_PW',
 'MFR_ENGINE_REDUCED_ROLLS',
 'ACFT_MFR_REDUCED_AIRBUS',
 'ACFT_MFR_REDUCED_ALENIA',
 'ACFT_MFR_REDUCED_BELL',
 'ACFT_MFR_REDUCED_BOEING',
 'ACFT_MFR_REDUCED_BOMBARDIER',
 'ACFT_MFR_REDUCED_CANADAIR',
 'ACFT_MFR_REDUCED_CHAMPION',
 'ACFT_MFR_REDUCED_CSERIES',
 'ACFT_MFR_REDUCED_EMBRAER',
 'ACFT_MFR_REDUCED_MCDONNELL',
 'ACFT_MFR_REDUCED_OTHER',
 'ACFT_MFR_REDUCED_PIPER',
 'ACFT_MFR_REDUCED_SAAB',
 'ACFT_MODEL_REDUCED_717_200',
 'ACFT_MODEL_REDUCED_737',
 'ACFT_MODEL_REDUCED_757',
 'ACFT_MODEL_REDUCED_767',
 'ACFT_MODEL_REDUCED_777',
 'ACFT_MODEL_REDUCED_787',
 'ACFT_MODEL_REDUCED_A319',
 'ACFT_MODEL_REDUCED_A320',
 'ACFT_MODEL_REDUCED_A321',
 'ACFT_MODEL_REDUCED_A330',
 'ACFT_MODEL_REDUCED_ATR42_500',
 'ACFT_MODEL_REDUCED_BD500',
 'ACFT_MODEL_REDUCED_CL600',
 'ACFT_MODEL_REDUCED_DHC8',
 'ACFT_MODEL_REDUCED_EMB135KL',
 'ACFT_MODEL_REDUCED_EMB145',
 'ACFT_MODEL_REDUCED_ERJ170',
 'ACFT_MODEL_REDUCED_ERJ190',
 'ACFT_MODEL_REDUCED_MD88',
 'ACFT_MODEL_REDUCED_MD90_30',
 'ACFT_MODEL_REDUCED_OTHER']]

In [ ]:
scaled_data_final.shape

In [ ]:
scaled_data_final.head()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
principalComponents = pca.fit_transform(scaled_data_final)
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Explained Variance')
plt.show()

From the diagram above, it can be seen that 50 principal components explain almost 90% of the variance in data and 60 principal components explain around 95% of the variance in data.

So, instead of giving all the columns as input, we’d only feed these 50 principal components of the data to the machine learning algorithm and we’d obtain a similar result.

In [ ]:
pca = PCA(n_components=50)
new_data = pca.fit_transform(scaled_data_final)
# This will be the new data fed to the algorithm.
principal_Df = pd.DataFrame(data = new_data
             , columns = ['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10',
                          'pc11','pc12','pc13','pc14','pc15','pc16','pc17','pc18','pc19','pc20',
                          'pc21','pc22','pc23','pc24','pc25','pc26','pc27','pc28','pc29','pc30',
                         'pc31','pc32','pc33','pc34','pc35','pc36','pc37','pc38','pc39','pc40',
                         'pc41','pc42','pc43','pc44','pc45','pc46','pc47','pc48','pc49','pc50'])

In [ ]:
principal_Df.describe()

In [ ]:
#principal_Df['DEP_DELAY']=base_dummied['DEP_DELAY']

In [ ]:
principal_Df.info()

In [ ]:
principal_Df.describe()

# REGRESSION MODEL ON PCA DATA

In [ ]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm

In [ ]:
base_dummied_regression_Y.shape

In [ ]:
principal_Df.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x=principal_Df
y=base_dummied_regression_Y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

In [ ]:
y_pred=regr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

In [ ]:
MAE=mean_absolute_error(y_test, y_pred)
MAE

In [ ]:
MSE=mean_squared_error(y_test, y_pred)
MSE

In [ ]:
RMSE=math.sqrt(MSE)
RMSE

In [ ]:
R2=r2_score(y_test, y_pred)
R2

In [ ]:
regr.score(x_test, y_test)

In [ ]:
regr.score(x_train, y_train)

# RANDOM FOREST MODEL ON PCA DATA


In [ ]:
def create_categories(y):
    if y<15 :
        return "NO DELAY"
    else:
        if y<20:
            return "15MIN"
        else:
            if y<35:
                return "30MIN"
            else:
                if y<50:
                    return "45MIN"
                else:
                    return "60MIN"
    

In [ ]:
create_categories(19)

In [ ]:
y_new=y.apply(create_categories)

In [ ]:
type(y_new)

In [ ]:
y_new=pd.DataFrame(y_new)
y_new["Count"]=1
y_new

In [ ]:
y_new.groupby(by="DEP_DELAY")["Count"].count()

In [ ]:
y_new.drop(columns=["Count"],inplace=True)
y_new

In [ ]:
x

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_new, test_size=0.2, random_state=4,stratify=y_new)

In [ ]:
rf = RandomForestClassifier(n_estimators=30, max_depth=15, random_state=123, n_jobs=-1)
rf.fit(x_train, y_train)

In [ ]:
# Gini importance
sorted_index = rf.feature_importances_.argsort()
plt.figure(figsize=[10,10])
plt.barh(x_train.columns[sorted_index].tolist()[:], rf.feature_importances_[sorted_index][:])
plt.xlabel("Feature Importance")

In [ ]:
y_pred_rf=rf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
rf_accuracy=accuracy_score(y_test, y_pred_rf)
rf_accuracy

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_test, y_pred_rf, average='macro')

In [ ]:
f1_score(y_test, y_pred_rf, average='micro')

In [ ]:
f1_score(y_test, y_pred_rf, average='weighted')

In [ ]:
f1_score(y_test, y_pred_rf, average=None)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_pred_proba=rf.predict_proba(x_test)
y_pred_proba

In [ ]:
y_test_new=pd.get_dummies(y_test)
y_test_new

In [ ]:
roc_auc_score(y_test_new, rf.predict_proba(x_test))

In [ ]:
from sklearn.metrics import roc_curve, auc

y_score_rf = y_pred_proba
n_classes_rf = y_test_new.shape[1]
n_classes_rf

In [ ]:
y_score_rf

In [ ]:
y_test_new1=np.asarray(y_test_new)
y_test_new1

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes_rf):
    fpr[i], tpr[i], _ = roc_curve(y_test_new1[:, i], y_score_rf[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_new1.ravel(), y_score_rf.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

### Logistic Regression

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


In [ ]:
lgr = LogisticRegression(random_state=0).fit(x_train, y_train)
lgr.predict(x_test)

#clf.predict_proba()

#clf.score(x, y)

In [ ]:
y_pred_proba_lgr=clf.predict_proba(x_test)
y_pred_proba_lgr

In [ ]:
y_score_lgr = y_pred_proba_clf
n_classes_lgr = y_test_new.shape[1]
n_classes_lgr

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes_lgr):
    fpr[i], tpr[i], _ = roc_curve(y_test_new1[:, i], y_score_lgr[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_new1.ravel(), y_score_lgr.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# ENSEMBLE OF RF and LG

In [ ]:
y_score_lgr

In [ ]:
y_score_rf

In [ ]:
y_score_ensemble=(y_score_lgr + y_score_rf)/2
y_score_ensemble

In [ ]:
n_classes_ens= y_test_new.shape[1]
n_classes_ens

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes_lgr):
    fpr[i], tpr[i], _ = roc_curve(y_test_new1[:, i], y_score_ensemble[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_new1.ravel(), y_score_ensemble.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# NEURAL NETWORK

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
x_train.shape

In [ ]:
import tensorflow as tf

# Prepare data.
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a base model -- sequential, functional, or subclass.
model = tf.keras.Sequential([
    tf.keras.Input((511678, 50), name='feature'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(5, activation=tf.nn.softmax)
])

# Compile, train, and evaluate.
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
model.fit({'feature': x_train, 'label': y_train}, batch_size=32, epochs=5)
model.evaluate({'feature': x_test, 'label': y_test})